Random Forestを実行してみる。

In [1]:
import pandas as pd
import sys

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import accuracy_score, mean_squared_error

print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


RandomForestはランダムに説明変数を引数`max_features`の数だけ持つ決定木を引数`n_estimator`の数だけ作り  
コンセンサスを取ることで精度を上げるアンサンブル機械学習である。`max_features`はデフォルトではすべての説明変数の数の平方根である。  
(例えばクラス分類の時、精度が50%以上の分類器を複数用意しコンセンサスを取ることで精度が増すことは数学的に証明されている)

### 1. RandomForestClassifier

[RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)を用いることでRandom Forestにてクラス分類を行うことができる。

#### 1.1 データの準備

Toyデータを用意する。  
[make_classification](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html#sklearn.datasets.make_classification)関数を使うと自身が望むようなクラス分類の検証のためのサンプルデータを用意することができる。  
今回は`サンプル数=1000`, `説明変数の数=10`, `クラス数=2` そして再現性を担保するために`random_state=0`, `shuffle=False`とした。

In [2]:
 X_clf, y_clf = make_classification(n_samples=1000, n_features=10, n_classes=2,
                                                random_state=0, shuffle=False)
X_clf = pd.DataFrame(X_clf)

In [3]:
X_clf.head()

,0,1,2,3,4,5,6,7,8,9
0,-1.668532,-1.299013,0.799353,-1.559985,-3.116857,0.644452,-1.913743,0.663562,-0.154072,1.193612
1,-2.972883,-1.088783,1.953804,-1.891656,-0.098161,-0.886614,-0.147354,1.059806,0.026247,-0.114335
2,-0.596141,-1.370070,-0.105818,-1.213570,0.743554,0.210359,-0.005927,1.366060,1.555114,0.613326
3,-1.068947,-1.175057,0.363982,-1.247739,-0.285959,1.496911,1.183120,0.718897,-1.216077,0.140672
4,-1.305269,-0.965926,0.647043,-1.183939,-0.743672,-0.159012,0.240057,0.100159,-0.475175,1.272954


In [4]:
y_clf[:5]

array([0, 0, 0, 0, 0])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [5]:
X_clf.shape

(1000, 10)

#### 1.2 機械学習

##### 1.2.1 RandomForestClassifierの使い方

決定木の数である`n_estimators=10`, 決定木の最大の深さ`max_depth=2`,  
そして再現性を担保するために`random_state=0`で機械学習を行う。

In [6]:
clf = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
clf.fit(X_clf, y_clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

`clf.feature_importances_`に各説明変数の重要度が保存されている。  
値が高ければ高いほど重要度が高い説明変数といえる。

In [7]:
print(clf.feature_importances_)

[ 0.05107855  0.44565534  0.09075854  0.36217254  0.00492064  0.
  0.00199749  0.04153638  0.00188052  0.        ]


予測には`predict`メソッドを用いる。  
例えば10個の説明変数がすべて0のサンプルの予測を行いたい時は以下のようにする。

In [8]:
print(clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

[1]


Training setの正答率を求めてみる。

[accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)関数を用いてTraining setの正答率を求めてみる。

In [9]:
y_predicted = clf.predict(X_clf)
accuracy_score(y_predicted, y_clf) * 100

95.399999999999991

正答率は約95.4%であった。

##### 1.2.2 決定木の数を増やす

決定木の数である`n_estimators`を100に増やして同様の解析を行ってみる。

In [10]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_clf, y_clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [11]:
print(clf.feature_importances_)

[ 0.02802468  0.46142492  0.12352124  0.34518939  0.00193351  0.0029199
  0.00777319  0.00947897  0.0108007   0.00893351]


In [12]:
y_predicted = clf.predict(X_clf)
accuracy_score(y_predicted, y_clf) * 100

95.799999999999997

正答率は95.8%であった。わずかであるが決定木の数を増やしよりコンセンサスを取りやすくすることで正答率があがった。

##### 1.2.3 決定木の複雑さを増やす

決定木の最大の深さである`max_depth`を3に増やして同様の解析を行ってみる。

In [13]:
clf = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=0)
clf.fit(X_clf, y_clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [14]:
y_predicted = clf.predict(X_clf)
accuracy_score(y_predicted, y_clf) * 100

96.099999999999994

正答率は約96.1%であった。わずかであるが決定木をより複雑にできるようにすることで正答率があがった。

ただし、冒頭でも伝えたようにRandomForestは複数の分類器によるコンセンサスで精度を上げるため  
一つ一つの分類機は弱学習器でよく、最大の深さである`max_depth`はあまり変化させる必要がない。

むしろ、以下のように決定木に使う説明変数の数である`max_features`の数を変化させて最適化することのほうが多い。

##### 1.2.4 決定木で用いる説明変数の数を変化させる

決定木に用いる説明変数の数である`max_features`の数を3に減らしてみる。  
`max_features`はデフォルトでは全ての説明変数の数の平方根である。

In [15]:
clf = RandomForestClassifier(n_estimators=10, max_features=3, random_state=0)
clf.fit(X_clf, y_clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [16]:
y_predicted = clf.predict(X_clf)
accuracy_score(y_predicted, y_clf) * 100

99.799999999999997

続けて、決定木に用いる説明変数の数である`max_features`の数を8にしてみる。  

In [17]:
clf = RandomForestClassifier(n_estimators=10, max_features=8, random_state=0)
clf.fit(X_clf, y_clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [18]:
y_predicted = clf.predict(X_clf)
accuracy_score(y_predicted, y_clf) * 100

99.700000000000003

今回の例では用いる説明変数の数を少なくすることで正答率があがった。  
用いる説明変数の数が多すぎると分類器に差がなくなることもあり単純に増やせばよいというわけではない。  
説明変数の数を色々変化させてcross validationを行うのが最も良い。

---

### 2. RandomForestRegressor

[RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)を用いることでRandom Forestにて回帰分析を行うことができる。

#### 2.1 データの準備

Toyデータを用意する。  
[make_regression](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html)関数を使うと自身が望むような回帰分析の検証のためのサンプルデータを用意することができる。  
今回は`サンプル数=1000`, `説明変数の数=10`, そして再現性を担保するために`random_state=0`, `shuffle=False`とした。

In [19]:
 X_regr, y_regr = make_regression(n_samples=1000, n_features=10,
                                                random_state=0, shuffle=False)
X_regr = pd.DataFrame(X_regr)

In [20]:
X_regr.head()

,0,1,2,3,4,5,6,7,8,9
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740


In [21]:
y_regr[:5]

array([ 300.2064792 ,  194.27686437,   13.25754564,  -24.6027897 ,
        -98.76061926])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [22]:
X_regr.shape

(1000, 10)

#### 2.2 機械学習

##### 2.2.1 RandomForestRegressorの使い方

決定木の数である`n_estimators=10`, 決定木の最大の深さ`max_depth=2`,  
そして再現性を担保するために`random_state=0`で機械学習を行う。

In [23]:
regr = RandomForestRegressor(n_estimators=10, max_depth=2, random_state=0)
regr.fit(X_regr, y_regr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

`clf.feature_importances_`に各説明変数の重要度が保存されている。  
値が高ければ高いほど重要度が高い説明変数といえる。

In [24]:
print(regr.feature_importances_)

[ 0.          0.          0.02265102  0.50486712  0.          0.
  0.31572423  0.15675763  0.          0.        ]


予測には`predict`メソッドを用いる。  
例えば10個の説明変数がすべて0のサンプルの予測を行いたい時は以下のようにする。

In [25]:
print(regr.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

[-32.17449828]


[mean_squared_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)関数を用いてTraining setのRoot Mean Square Error (RMSE) を求めてみる。

In [26]:
y_predicted = regr.predict(X_regr)
mean_squared_error(y_regr, y_predicted)

11343.859988077465

RMSEは約11343.9であった。

##### 2.2.2 決定木の数を増やす

決定木の数である`n_estimators`を100に増やして同様の解析を行ってみる。

In [27]:
regr = RandomForestRegressor(n_estimators=100, max_depth=2, random_state=0)
regr.fit(X_regr, y_regr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [28]:
print(regr.feature_importances_)

[ 0.          0.          0.02318644  0.46182728  0.          0.00220828
  0.30260993  0.21016807  0.          0.        ]


In [29]:
y_predicted = regr.predict(X_regr)
mean_squared_error(y_regr, y_predicted)

11328.125407195668

RMSEは約11328.1であった。わずかであるが決定木の数を増やしよりコンセンサスを取りやすくすることでRMSEが小さくなった。

##### 2.2.3 決定木の複雑さを増やす

決定木の最大の深さである`max_depth`を3に増やして同様の解析を行ってみる。

In [30]:
regr = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=0)
regr.fit(X_regr, y_regr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [31]:
y_predicted = regr.predict(X_regr)
mean_squared_error(y_regr, y_predicted)

8462.4899704699292

RMSEは約8386.7であった。今回のデータセットでは決定木をより複雑にできるようにすることで大幅にRMSEが小さくなった。

##### 2.2.4 決定木で用いる説明変数の数を変化させる

決定木に用いる説明変数の数である`max_features`の数を3に減らしてみる。  
`max_features`はデフォルトでは全ての説明変数の数の平方根である。

In [32]:
regr = RandomForestRegressor(n_estimators=10, max_features=3, random_state=0)
regr.fit(X_regr, y_regr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=3, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [33]:
y_predicted = regr.predict(X_regr)
mean_squared_error(y_regr, y_predicted)

970.52943197845366

続けて、決定木に用いる説明変数の数である`max_features`の数を8にしてみる。  

In [34]:
regr = RandomForestRegressor(n_estimators=10, max_features=8, random_state=0)
regr.fit(X_regr, y_regr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [35]:
y_predicted = regr.predict(X_regr)
mean_squared_error(y_regr, y_predicted)

873.40663827214928

今回の例では用いる説明変数の数を増やすことで正答率があがった。  
ただ、用いる説明変数の数が多すぎると分類器に差がなくなることもあり単純に増やせばよいというわけではない。  
説明変数の数を色々変化させてcross validationを行うのが最も良い。